In [2]:
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import json 
import requests

In [2]:
# connection string'

sqlitedb_rel_path = '../data/raw/raw_data_project_m1.db'
conn_str = f'sqlite:///{sqlitedb_rel_path}'

engine = create_engine(conn_str)

In [3]:
pd.read_sql_query("""
SELECT * FROM personal_info
;
""", engine)

,uuid,age,gender,dem_has_children,age_group
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,male,NO,40_65
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,male,nO,26_39
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,Male,YES,40_65
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,Fem,yES,40_65
...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,37 years old,FeMale,nO,26_39
9645,39f989f0-db52-0133-8482-0a81e8b09a82,53 years old,Male,yES,40_65
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,1992,male,NO,juvenile
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,47 years old,male,yES,40_65


In [4]:
pd.read_sql_query("""
SELECT * FROM country_info
;
""", engine)

,uuid,country_code,rural
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,countryside
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,city
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,Country
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,city
...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,SK,urban
9645,39f989f0-db52-0133-8482-0a81e8b09a82,SK,urban
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,SK,Non-Rural
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,SK,city


In [1]:
# Import initial data from the db:uuid , age and countries
initial_df = pd.read_sql_query("""
SELECT personal_info.uuid, normalized_job_code ,age , country_code FROM personal_info
INNER JOIN country_info ON personal_info.uuid = country_info.uuid
INNER JOIN career_info ON personal_info.uuid = career_info.uuid;
""", engine)

NameError: name 'pd' is not defined

In [6]:
# Import initial data from the db:uuid , age and countries
#trial_df = pd.read_sql_query("""
#SELECT country_code,normalized_job_code ,age   FROM personal_info
#INNER JOIN country_info ON personal_info.uuid = country_info.uuid
#INNER JOIN career_info ON personal_info.uuid = career_info.uuid
#GROUP BY country_code, normalized_job_code;
#""", engine)

In [7]:
#trial_df

In [8]:
#initial_df['uuid'] = initial_df['uuid'].str.replace(r'-', '')

In [9]:
def fix_age (age):
    if 'years old' in age:
        born_y = 2016 - int(re.sub("[^\d]", "", age)) 
    else:
        born_y = int(age)
        
    right_age = 2021 - born_y     
    
    return right_age 

initial_df['age'] = initial_df.apply(lambda x: fix_age(x['age']), axis=1 )

In [10]:
def group_age (age):
    if age <= 25:
         age_group = '(18-25)' 
    elif age <= 35:
        age_group = '[30-35)'
    elif age <= 45:
        age_group = '[40-45)'
    elif age <= 55:
        age_group = '[45-55)'
    else:
        age_group = '[55-70)'
    
    return age_group

initial_df['Age Group'] = initial_df.apply(lambda x: group_age(x['age']), axis=1 )

In [11]:
initial_df['age'].max()

70

In [12]:
job_code_unique = initial_df['normalized_job_code'].unique()

In [13]:
len(job_code_unique)


157

In [14]:
#job_code_unique = list(filter(None, job_code_unique))

In [15]:
job_code_unique[:5]

array([None, '861a9b9151e11362eb3c77ca914172d0',
       '049a3f3a2b5f85cb2971ba77ad66e10c',
       'f4b2fb1aa40f661488e2782b6d57ad2f',
       '27af8700f5577cec835acee2cb90a2ff'], dtype=object)

In [16]:
job_titles = {}
#code = "8a7a3f1bce0958ae9f090683e5a925b7"
for code in job_code_unique[1:]:
    results = requests.get(f'http://api.dataatwork.org/v1/jobs/{code}').json()
    job = results['title']
    job_titles[code] = job


In [17]:
job_titles

{'861a9b9151e11362eb3c77ca914172d0': 'Automatic Data Processing Planner',
 '049a3f3a2b5f85cb2971ba77ad66e10c': 'Data Coordinator',
 'f4b2fb1aa40f661488e2782b6d57ad2f': 'Database Developer',
 '27af8700f5577cec835acee2cb90a2ff': 'Data Entry Specialist',
 'c1b670eba9ccb65e7c99f7da116d5b9c': 'Database Architect',
 'd71c4164c99a5f9bca773e755a72f40f': 'Geographic Information Systems Database Administrator (GIS Database Administrator)',
 '2ee80cf30e2483dfe0947adc0428f8c5': 'Crime Data Specialist',
 '897ef18a9321616495831a4b4b0c1bb2': 'Data Communications Software Consultant',
 'a003702c0cea282e17967772dd83de5e': 'Data Security Analyst',
 '04c82eae29e1c4a30eac0710b74ab276': 'Database Marketing Analyst',
 'fcaf4aac737e53ad24b9f32f08b8740f': 'Computer or Data Processing Systems Consultant',
 '3833bd699e7afe4665b595f282da0387': 'Database Development and Administration Project Manager',
 '19e384e977796c25780d8c28b12089c4': 'Naval Tactical Data System--Combat Information Center Watch Officer, Gener

In [18]:
job_titles_df = pd.DataFrame(job_titles.items() ,columns=['normalized_job_code','Job Title'])


In [19]:
job_titles_df

In [27]:
url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html = requests.get(url).content
html[0:600]

b'\xef\xbb\xbf<!DOCTYPE html>\n<html lang="en" dir="ltr" class="client-nojs">\n<head>\n<meta http-equiv="X-UA-Compatible" content="Ie=Edge,chrome=1"/> <meta charset="UTF-8" /><title>Glossary:Country codes - Statistics Explained</title>\n<meta name="generator" content="MediaWiki 1.22.5" />\n<link rel="shortcut icon" href="https://ec.europa.eu/eurostat/estat-theme/images/favicon.ico" />\n<link rel="search" type="application/opensearchdescription+xml" href="/eurostat/statistics-explained/opensearch_desc.php" title="Statistics Explained (en)" />\n<link rel="EditURI" type="application/rsd+xml" href="https://ec.europ'

In [28]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(html, "html.parser")
soup

In [55]:
table_countries = soup.find('table',{'width':'80%'}).text
table_countries

'\n\nBelgium\n\n(BE)\n\nGreece\n\n(EL)\n\nLithuania\n\n(LT)\n\nPortugal\n\n(PT)\n\n\nBulgaria\n\n(BG)\n\nSpain\n\n(ES)\n\nLuxembourg\n\n(LU)\n\nRomania\n\n(RO)\n\n\nCzechia\n\n(CZ)\n\nFrance\n\n(FR)\n\nHungary\n\n(HU)\n\nSlovenia\n\n(SI)\n\n\nDenmark\n\n(DK)\n\nCroatia\n\n(HR)\n\nMalta\n\n(MT)\n\nSlovakia\n\n(SK)\n\n\nGermany\n\n(DE)\n\nItaly\n\n(IT)\n\nNetherlands\n\n(NL)\n\nFinland\n\n(FI)\n\n\nEstonia\n\n(EE)\n\nCyprus\n\n(CY)\n\nAustria\n\n(AT)\n\nSweden\n\n(SE)\n\n\nIreland\n\n(IE)\n\nLatvia\n\n(LV)\n\nPoland\n\n(PL)\n'

In [56]:
table_countries = table_countries.replace("\n","")

In [64]:
table_countries

['Belgium(BE',
 'Greece(EL',
 'Lithuania(LT',
 'Portugal(PT',
 'Bulgaria(BG',
 'Spain(ES',
 'Luxembourg(LU',
 'Romania(RO',
 'Czechia(CZ',
 'France(FR',
 'Hungary(HU',
 'Slovenia(SI',
 'Denmark(DK',
 'Croatia(HR',
 'Malta(MT',
 'Slovakia(SK',
 'Germany(DE',
 'Italy(IT',
 'Netherlands(NL',
 'Finland(FI',
 'Estonia(EE',
 'Cyprus(CY',
 'Austria(AT',
 'Sweden(SE',
 'Ireland(IE',
 'Latvia(LV',
 'Poland(PL',
 '']

In [58]:
table_countries = table_countries.split(")")

In [68]:
list_countries = [i.split('(') for i in table_countries] 
list_countries

[['Belgium', 'BE'],
 ['Greece', 'EL'],
 ['Lithuania', 'LT'],
 ['Portugal', 'PT'],
 ['Bulgaria', 'BG'],
 ['Spain', 'ES'],
 ['Luxembourg', 'LU'],
 ['Romania', 'RO'],
 ['Czechia', 'CZ'],
 ['France', 'FR'],
 ['Hungary', 'HU'],
 ['Slovenia', 'SI'],
 ['Denmark', 'DK'],
 ['Croatia', 'HR'],
 ['Malta', 'MT'],
 ['Slovakia', 'SK'],
 ['Germany', 'DE'],
 ['Italy', 'IT'],
 ['Netherlands', 'NL'],
 ['Finland', 'FI'],
 ['Estonia', 'EE'],
 ['Cyprus', 'CY'],
 ['Austria', 'AT'],
 ['Sweden', 'SE'],
 ['Ireland', 'IE'],
 ['Latvia', 'LV'],
 ['Poland', 'PL'],
 ['']]

In [81]:
country_codes_df=pd.DataFrame(list_countries, columns=('Country','country_code'))

In [82]:
country_codes_df

In [20]:
final_df = pd.merge(initial_df,  
                      job_titles_df,  
                      on ='normalized_job_code',  
                      how ='inner') 

In [87]:
definitive_df = pd.merge(final_df,  
                      country_codes_df,  
                      on ='country_code',  
                      how ='inner') 

In [117]:
definitive_df

,uuid,normalized_job_code,age,country_code,Age Group,Job Title,Country
0,54f0f1c0-dda1-0133-a559-0a81e8b09a82,861a9b9151e11362eb3c77ca914172d0,62,AT,[55-70),Automatic Data Processing Planner,Austria
1,cf24ed60-da3f-0133-0034-0a81e8b09a82,861a9b9151e11362eb3c77ca914172d0,53,AT,[45-55),Automatic Data Processing Planner,Austria
2,15626d40-db13-0133-ea5c-0a81e8b09a82,049a3f3a2b5f85cb2971ba77ad66e10c,50,AT,[45-55),Data Coordinator,Austria
3,ca3f65f0-d92b-0133-3959-0a81e8b09a82,049a3f3a2b5f85cb2971ba77ad66e10c,51,AT,[45-55),Data Coordinator,Austria
4,24954a70-db98-0133-4a64-0a81e8b09a82,f4b2fb1aa40f661488e2782b6d57ad2f,46,AT,[45-55),Database Developer,Austria
...,...,...,...,...,...,...,...
4921,6e8c05a0-d98b-0133-a515-0a81e8b09a82,a12a16280e22220f9354333f6fe1b22a,36,MT,[40-45),Geographic Information Systems Data Administra...,Malta
4922,46f91e60-d8cc-0133-611d-0a81e8b09a82,970eb38c7b3eb080466052ea994af189,47,MT,[45-55),Internet Database Specialist,Malta
4923,0c5e7600-d9ad-0133-8817-0a81e8b09a82,7bb15eefb19ed09c59349baad7e8469e,36,MT,[40-45),Database Coordinator,Malta
4924,f9003a20-dbc6-0133-9c2b-0a81e8b09a82,7bb15eefb19ed09c59349baad7e8469e,47,MT,[45-55),Database Coordinator,Malta


In [112]:
grouped_df = definitive_df[['uuid','Country', 'Job Title','Age Group']].groupby(['Country', 'Job Title','Age Group']).count().reset_index()

In [113]:
grouped_df

,Country,Job Title,Age Group,uuid
0,Austria,Automatic Data Processing Customer Liaison (AD...,[40-45),2
1,Austria,Automatic Data Processing Planner,[45-55),1
2,Austria,Automatic Data Processing Planner,[55-70),1
3,Austria,Automatic Data Processing Systems Security Spe...,[30-35),1
4,Austria,Automatic Data Processing Systems Security Spe...,[45-55),1
...,...,...,...,...
3040,Sweden,Head of Information Systems and Data Processing,[45-55),1
3041,Sweden,Naval Tactical Data System--Combat Information...,[30-35),1
3042,Sweden,Oracle Database Administrator (Oracle DBA),[40-45),1
3043,Sweden,SQL Database Administrator,[45-55),1


In [114]:
grouped_df.rename(columns={'uuid':'Quantity'}, inplace=True)
grouped_df[:20]

,Country,Job Title,Age Group,Quantity
0,Austria,Automatic Data Processing Customer Liaison (AD...,[40-45),2
1,Austria,Automatic Data Processing Planner,[45-55),1
2,Austria,Automatic Data Processing Planner,[55-70),1
3,Austria,Automatic Data Processing Systems Security Spe...,[30-35),1
4,Austria,Automatic Data Processing Systems Security Spe...,[45-55),1
5,Austria,Clinical Data Specialist,[40-45),1
6,Austria,Computer or Data Processing Systems Consultant,[30-35),1
7,Austria,Computer or Data Processing Systems Consultant,[45-55),1
8,Austria,Computer or Data Processing Systems Consultant,[55-70),1
9,Austria,Crime Data Specialist,[45-55),1


In [116]:
grouped_df.to_csv('../data/countries.csv', index=False)

In [118]:
grouped_df = definitive_df[['uuid','Country', 'Job Title','Age Group']].groupby(['Country', 'Job Title','Age Group']).count()
grouped_df.rename(columns={'uuid':'Quantity'}, inplace=True)
grouped_df['Percentage'] = grouped_df.groupby(level=0).transform(lambda x: x/x.sum())

In [120]:
grouped_df.reset_index()

,Country,Job Title,Age Group,Quantity,Percentage
0,Austria,Automatic Data Processing Customer Liaison (AD...,[40-45),2,0.027027
1,Austria,Automatic Data Processing Planner,[45-55),1,0.013514
2,Austria,Automatic Data Processing Planner,[55-70),1,0.013514
3,Austria,Automatic Data Processing Systems Security Spe...,[30-35),1,0.013514
4,Austria,Automatic Data Processing Systems Security Spe...,[45-55),1,0.013514
...,...,...,...,...,...
3040,Sweden,Head of Information Systems and Data Processing,[45-55),1,0.010638
3041,Sweden,Naval Tactical Data System--Combat Information...,[30-35),1,0.010638
3042,Sweden,Oracle Database Administrator (Oracle DBA),[40-45),1,0.010638
3043,Sweden,SQL Database Administrator,[45-55),1,0.010638
